# PostgreSQLを操作する

In [1]:
from dotenv import load_dotenv
from psycopg_pool import ConnectionPool
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
import os

In [2]:
load_dotenv()

conninfo = os.environ["POSTGRES_CONNINFO"]
url = os.environ["SQLARCHEMY_URL"]

In [3]:
with ConnectionPool(conninfo=conninfo) as pool:
    with pool.connection() as session:
        with session.execute("select * from messages order by id asc") as cur:
            for record in cur:
                print(record)

(1, 'foo')
(2, 'bar')
(3, 'baz')
(4, 'qux')


## SQLAlchemy

In [4]:
engine = create_engine(url)

# engine.connect()でもデータベースの操作はできるけれど、ORM的な機能を使うにはSessionを使うみたい。
# この例では単純にクエリーを投げているだけなのでConnectionを使うのと変わらない使い方になってる。
#with engine.connect() as conn:
with Session(engine) as session:
    result = session.execute(text("select * from messages order by id asc"))
    print(result.all())

    result = session.execute(text("select * from messages where id = :id"), { "id": "1" })
    # 動的にカラム名が付けられたタプルがイテレートされる
    print([row.content for row in result.all()])

    session.execute(
        text("insert into messages (content) values (:content)"), 
        [{ "content": "hoge" }, { "content": "fuga" }],
    )
    #conn.commit()
    result = session.execute(text("select content from messages order by id asc"))
    # content, でタプルを分解して受け取っている
    print([content for content, in result.all()])



[(1, 'foo'), (2, 'bar'), (3, 'baz'), (4, 'qux')]
['foo']
['foo', 'bar', 'baz', 'qux', 'hoge', 'fuga']
